## **Potential Outcomes and RCTs**

# Data Simulation
Simulate a dataset with (n = 1000) individuals. Generate:
*   Covariates $X_1$, $X_2$, $X_3$, $X_4$ (continuous or binary)
*   Treatment assignment $D \sim \text{Bernoulli}(0.5)$
*   Outcome variable:
$Y = 2D + 0.5X_1 - 0.3X_2 + 0.2X_3 + \epsilon, \quad \epsilon \sim N(0, 1)$

Save everything in a `data.frame`.

Perform a balance check: compare the means of ($X_1$, $X_2$, $X_3$, $X_4$) across treatment and control groups (e.g., using `t.test` or regression).

In [ ]:
# Instalar paquetes
using Pkg
Pkg.add("Distributions")
Pkg.add("GLM")
Pkg.add("DataFrames")
Pkg.add("Statistics")

# Importar librerías
using Distributions
using GLM
using DataFrames
using Statistics
using Random

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
    Updating `~/.julia/environments/v1.11/Project.toml`
  [31c24e10] + Distributions v0.25.120
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
   Installed ShiftedArrays ─ v2.0.0
   Installed GLM ─────────── v1.9.0
   Installed StatsModels ─── v0.7.7
    Updating `~/.julia/environments/v1.11/Project.toml`
  [38e38edf] + GLM v1.9.0
    Updating `~/.julia/environments/v1.11/Manifest.toml`
  [38e38edf] + GLM v1.9.0
  [1277b4bf] + ShiftedArrays v2.0.0
  [3eaba693] + StatsModels v0.7.7
Precompiling project...
   1828.5 ms  ✓ ShiftedArrays
   4349.1 ms  ✓ StatsModels
   5090.2 ms  ✓ GLM
  3 dependencies successfully precompiled in 13 seconds. 497 already precompiled.
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
    Upd

In [ ]:
# Data Simulation
Random.seed!(42)
n = 1000

# Generar covariables
X1 = rand(Normal(0, 1), n)
X2 = rand(Normal(0, 1), n)
X3 = rand(Normal(0, 1), n)
X4 = rand(Normal(0, 1), n)

# Tratamiento (Bernoulli)
D = rand(Binomial(1, 0.5), n)

# Término de error
epsilon = rand(Normal(0, 1), n)

# Outcome variable
Y = 2 .* D + 0.5 .* X1 - 0.3 .* X2 + 0.2 .* X3 + epsilon

# Data frame
df = DataFrame(Y=Y, D=D, X1=X1, X2=X2, X3=X3, X4=X4)
df

Row,Y,D,X1,X2,X3,X4
,Float64,Int64,Float64,Float64,Float64,Float64
1,2.77694,1,-0.363357,-0.01988,1.34286,-0.943026
2,2.25155,1,0.251737,0.218246,-0.996272,1.23951
3,3.4161,1,-0.314988,1.52856,0.573589,1.45273
4,0.321127,0,-0.311252,-0.764316,0.167796,-0.886181
5,1.00886,0,0.816307,-2.02765,0.109283,2.16943
6,-1.05376,0,0.476738,1.02999,-0.531441,-0.148483
7,-0.810991,0,-0.859555,0.981408,0.101455,-0.94444
8,2.98603,1,-1.46929,-0.513191,1.75233,0.619433
9,0.926849,1,-2.11433,0.613236,0.684612,-1.02141


In [ ]:
# Prueba de balance
println("Balance Check")
println("="^60)
println(lpad("Variable", 10) * lpad("Tratamiento", 12) * lpad("Control", 12) * lpad("Diferencia", 12) * lpad("p-value", 12))
println("-"^60)

covariate_names = [:X1, :X2, :X3, :X4]

for cov_name in covariate_names
    mean_treated = mean(df[df.D .== 1, cov_name])
    mean_control = mean(df[df.D .== 0, cov_name])
    diff = mean_treated - mean_control
    cov_data = df[:, cov_name] .- mean(df[:, cov_name])
    D_data = df.D .- mean(df.D)
    model = lm([D_data;;], cov_data)

    coeftable_result = coeftable(model)
    p_value = coeftable_result.cols[4][1]

    println(lpad(string(cov_name), 10) *
            lpad(round(mean_treated, digits=4), 12) *
            lpad(round(mean_control, digits=4), 12) *
            lpad(round(diff, digits=4), 12) *
            lpad(round(p_value, digits=4), 12))
end

Balance Check
  Variable Tratamiento     Control  Diferencia     p-value
------------------------------------------------------------
        X1     -0.0551     -0.0654      0.0104      0.8683
        X2      0.0523     -0.0158      0.0681       0.289
        X3       0.047      0.0031      0.0439      0.4802
        X4      0.0084     -0.0428      0.0512      0.4283


# Estimating the Average Treatment Effect
Estimate the treatment effect (ATE) using a simple regression:
$Y \sim D$

Estimate the ATE controlling for all covariates:
$Y \sim D + X_1 + X_2 + X_3 + X_4$

Compare the two estimates. Answer the following:
*   Does the ATE change?
*   What happens to the standard errors?

In [ ]:
# Estimación del ATE
println("\n" * "="^60)
println("Average Treatment Effect Estimation")
println("="^60)

# Modelo 1: Regresión simple Y ~ D
println("Model 1: Y ~ D")
model1 = lm(@formula(Y ~ D), df)
coef_table1 = coeftable(model1)

results_df1 = DataFrame(
    Variable = coef_table1.rownms,
    Coef = round.(coef_table1.cols[1], digits=4),
    Std_Error = round.(coef_table1.cols[2], digits=4),
    t_value = round.(coef_table1.cols[3], digits=4),
    p_value = round.(coef_table1.cols[4], digits=4)
)

println("Modelo 1 - Y ~ D:")
println(results_df1)
println("\n" * "-"^40)

# Modelo 2: Regresión con covariables
println("Model 2: Y ~ D + X1 + X2 + X3 + X4")
model2 = lm(@formula(Y ~ D + X1 + X2 + X3 + X4), df)
coef_table2 = coeftable(model2)

results_df2 = DataFrame(
    Variable = coef_table2.rownms,
    Coef = round.(coef_table2.cols[1], digits=4),
    Std_Error = round.(coef_table2.cols[2], digits=4),
    t_value = round.(coef_table2.cols[3], digits=4),
    p_value = round.(coef_table2.cols[4], digits=4)
)

println("Modelo 2 - Y ~ D + X1 + X2 + X3 + X4:")
println(results_df2)


Average Treatment Effect Estimation
Model 1: Y ~ D
Modelo 1 - Y ~ D:
2×5 DataFrame
 Row │ Variable     Coef     Std_Error  t_value  p_value
     │ String       Float64  Float64    Float64  Float64
─────┼───────────────────────────────────────────────────
   1 │ (Intercept)  -0.0579     0.0519  -1.1154    0.265
   2 │ D             1.9813     0.0725  27.3211    0.0

----------------------------------------
Model 2: Y ~ D + X1 + X2 + X3 + X4
Modelo 2 - Y ~ D + X1 + X2 + X3 + X4:
6×5 DataFrame
 Row │ Variable     Coef     Std_Error  t_value   p_value
     │ String       Float64  Float64    Float64   Float64
─────┼────────────────────────────────────────────────────
   1 │ (Intercept)  -0.0317     0.0441   -0.72     0.4717
   2 │ D             1.9861     0.0615   32.2858   0.0
   3 │ X1            0.4721     0.0311   15.1661   0.0
   4 │ X2           -0.3148     0.0303  -10.3915   0.0
   5 │ X3            0.2392     0.0313    7.6464   0.0
   6 │ X4            0.0241     0.0301    0.7993  

## Comparación de las Estimaciones del ATE

### ¿Cambia el ATE?

Sí, el ATE cambia ligeramente:
- **Modelo 1** (sin controles): ATE = 1.9813
- **Modelo 2** (con controles): ATE = 1.9861

La diferencia es de aproximadamente 0.0048. El ATE aumenta ligeramente cuando controlamos por las covariables, acercándose más al valor real de 2.0 que usamos en la simulación de datos.

### ¿Qué pasa con los errores estándar?

Los errores estándar mejoran (disminuyen).
- **Modelo 1**: std err = 0.0725
- **Modelo 2**: std err = 0.0615

El error estándar se reduce en aproximadamente 15%. Esto indica que:

1. Mayor precisión: La estimación del ATE es más precisa cuando incluimos controles
2. Intervalos de confianza más estrechos:
   - Modelo 1: [1.839, 2.124]
   - Modelo 2: [1.865, 2.107]
3. Menor variabilidad: Los controles explican parte de la varianza residual en Y

### Conclusión

Incluir covariables relevantes (especialmente X₁, X₂, X₃, que son predictores de Y) no solo mejora la precisión del estimador, sino que también corrige ligeramente el sesgo de asignación, acercando el ATE estimado al valor verdadero del efecto causal.

# LASSO and Variable Selection


- Use `cv.glmnet` to fit a LASSO model of ($Y$) on the covariates \($X_1$, ..., $X_q$), excluding the treatment.  
  - Report which covariates are selected at ($\lambda_{min}$).

- Re-estimate the ATE with only the covariates selected by LASSO:  

  \[
  $Y \sim D + X_{selected}$
  \]

- Compare this estimate with those from Part B. Discuss whether the accuracy changes and what advantages using LASSO might have in this context.


In [ ]:
using Pkg
Pkg.add("StatsModels")

   Resolving package versions...
    Updating `~/.julia/environments/v1.11/Project.toml`
  [3eaba693] + StatsModels v0.7.7
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`


In [ ]:
using GLMNet
using GLM
using DataFrames
using Statistics
using StatsModels

In [ ]:
println("LASSO variable selection (excluye D)")
println("="^60)

# solo covariables (excluye D) y outcome
covariate_syms = [:X1, :X2, :X3, :X4]
X = Matrix(df[:, covariate_syms])
y = df.Y

# estandarización (penalización simétrica)
X_std = (X .- mean(X, dims=1)) ./ std(X, dims=1)

# LASSO
cv_lasso = glmnetcv(X_std, y, alpha=1.0)  # nfolds=10 por defecto
idx_min = argmin(cv_lasso.meanloss)
lambda_min = cv_lasso.lambda[idx_min]
betas_min = cv_lasso.path.betas[:, idx_min]  # sin intercepto

# selección de variables
selected_mask = abs.(betas_min) .> 1e-10 # (umbral pequeño)
selected_vars = String.(covariate_syms[selected_mask])

println("λ_min:", lambda_min)
println("Covariables seleccionadas:", isempty(selected_vars) ? "Ninguna" : join(selected_vars, ", "))

# re-estimar ATE: Y ~ D + X_selected (OLS)
if isempty(selected_vars)
    fml = @formula(Y ~ D)
else
    # Construir fórmula dinámicamente con StatsModels.Term
    terms = Term.(Symbol.(selected_vars))
    fml = Term(:Y) ~ Term(:D) + sum(terms)
end

model3 = lm(fml, df)


println("\n", "="^60)
println("ATE con covariables seleccionadas (OLS)")
print(coeftable(model3))


LASSO variable selection (excluye D)
λ_min:0.0019581205870411035
Covariables seleccionadas:X1, X2, X3, X4

ATE con covariables seleccionadas (OLS)
─────────────────────────────────────────────────────────────────────────────
                  Coef.  Std. Error       t  Pr(>|t|)   Lower 95%   Upper 95%
─────────────────────────────────────────────────────────────────────────────
(Intercept)  -0.0317374   0.0440796   -0.72    0.4717  -0.118237    0.0547625
D             1.98609     0.061516    32.29    <1e-99   1.86538     2.10681
X1            0.472135    0.031131    15.17    <1e-46   0.411045    0.533225
X2           -0.314794    0.0302934  -10.39    <1e-23  -0.374241   -0.255348
X3            0.239177    0.0312796    7.65    <1e-13   0.177796    0.300559
X4            0.0240627   0.0301049    0.80    0.4243  -0.0350137   0.0831391
─────────────────────────────────────────────────────────────────────────────

El ATE estimado con LASSO (≈1.99) es prácticamente igual al obtenido con todas las covariables en el Modelo 2 (≈2.04). Vemos que LASSO retiene X4, aún siendo irrelevante en el DGP. Esto refleja que con λ_min la penalización es débil. Sin embargo, esto no afecta la estimación principal del ATE, que sigue siendo precisa y cercana al valor verdadero.